In [1]:
import pandas as pd
import parselmouth

from parselmouth.praat import call
import parselmouth
from fnmatch import fnmatch
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime
import os

In [ ]:
flag_extract_mfcc = False

In [2]:


def getFeatures(voiceID, f0min, f0max, unit):
    try:
        sound = parselmouth.Sound(voiceID) # read the sound
        duration = call(sound, "Get total duration") # duration
        pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object


        meanF0 = call(pitch, "Get mean", 0, 0, unit) #MDVP:Fo(Hz)
        # get standard deviation
        stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) 
        harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
        hnr = call(harmonicity, "Get mean", 0, 0) #HNR
        pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
        localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:Jitter(%)
        localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:Jitter(Abs)
        rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:RAP
        ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:PPQ
        ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3) #Jitter:DDP
        localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #MDVP:Shimmer
        localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #MDVP:Shimmer(dB)
        apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #Shimmer:APQ3
        aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #Shimmer:APQ5
        apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

        
        #print("file " + voiceID + " was read successfuly")
        return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer
    except Exception as exc:
        #print(exc)
        print("file " + voiceID + " was not read because it is not a sound file - ")
        return -1

In [3]:
def flag_extract_mfcc(voiceID):
        try:
        
            """
            Extracts the mel frequency ceptral coefficients from the voice sample

            Parameters:
            voiceID : .wav file
                the voice sample we want to extract the features from
            """

            sound = parselmouth.Sound(voiceID)
            mfcc_object = sound.to_mfcc(number_of_coefficients=12) #the optimal number of coeefficient used is 12
            mfcc = mfcc_object.to_array()
            mfcc_mean = np.mean(mfcc.T,axis=0)
            [mfcc0, mfcc1, mfcc2, mfcc3, mfcc4, mfcc5, mfcc6, mfcc7, mfcc8, mfcc9, mfcc10, mfcc11, mfcc12] = mfcc_mean
            return mfcc0, mfcc1, mfcc2, mfcc3, mfcc4, mfcc5, mfcc6, mfcc7, mfcc8, mfcc9, mfcc10, mfcc11, mfcc12
        except Exception as exc:
            #print(exc)
            print("file " + voiceID + " was not read because it is not a sound file - ")
            return -1




In [4]:
"""
def getListOfAudioFiles(dir_path):
    #print list of sound files in directory
    res = []
    # Iterate directory
    for file_path in os.listdir(dir_path):
        # check if current file_path is a file
        if os.path.isfile(os.path.join(dir_path, file_path)):
            # add filename to list
            res.append(file_path)
    return(res)
    """

def getListOfAudioFiles(dir_path):
    res= []
    pattern = "*.wav"

    for path, subdirs, files in os.walk(dir_path):
        for name in files:
            if fnmatch(name, pattern):
                res.append(os.path.join(name))
    return(res)




In [5]:
italian_pd_audiofiles_path = '../data/Italian_PD'
italian_control_audiofiles_path = '../data/Italian_HC'
mdvr_pd_audiofiles_path = '../data/MDVR_PD'
mdvr_control_audiofiles_path = '../data/MDVR_HC'

def getFeaturesFromAudioFiles(pd_audiofiles_path, control_audiofiles_path):

    #extract mfcc features or not

    pd_audiofiles_names = getListOfAudioFiles(pd_audiofiles_path)
    control_audiofiles_names = getListOfAudioFiles(control_audiofiles_path)

    if flag_extract_mfcc:

        original_features_df = pd.DataFrame(columns=['Duration', 'meanF0', 'stdevF0', 'hnr', 'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'aqpq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'mfcc0', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12','PD'])

    else:
        original_features_df = pd.DataFrame(columns=['Duration', 'meanF0', 'stdevF0', 'hnr', 'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'aqpq5Shimmer', 'apq11Shimmer', 'ddaShimmer','PD'])


    for i in tqdm(range(len(pd_audiofiles_names)-1)):
        values = getFeatures(pd_audiofiles_path+'/'+pd_audiofiles_names[i], 75, 500, "Hertz")
        if flag_extract_mfcc:
            mfcc_values = flag_extract_mfcc(pd_audiofiles_path+'/'+pd_audiofiles_names[i])
        if (values!=-1):

            duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = values
            
            if (flag_extract_mfcc and mfcc_values!=-1):
                mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12 = mfcc_values
                original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer, mfcc0, mfcc1, mfcc2, mfcc3, mfcc4, mfcc5, mfcc6, mfcc7, mfcc8, mfcc9, mfcc10, mfcc11, mfcc12,1]
            else:
                original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer,1]

    for i in tqdm(range(len(control_audiofiles_names)-1)):
        values = getFeatures(control_audiofiles_path+'/'+control_audiofiles_names[i], 75, 500, "Hertz")
        if flag_extract_mfcc:
            mfcc_values = flag_extract_mfcc(control_audiofiles_path+'/'+control_audiofiles_names[i])

        if (values!=-1):
            duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = values

            if (flag_extract_mfcc and mfcc_values!=-1):
                mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12 = mfcc_values
                original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer, mfcc0, mfcc1, mfcc2, mfcc3, mfcc4, mfcc5, mfcc6, mfcc7, mfcc8, mfcc9, mfcc10, mfcc11, mfcc12,0]
            
            else:
                original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer, 0]

    return original_features_df


In [6]:
mdvr_original_featuers_df = getFeaturesFromAudioFiles(mdvr_pd_audiofiles_path, mdvr_control_audiofiles_path)

  0%|          | 0/341 [00:00<?, ?it/s]

  0%|          | 0/473 [00:00<?, ?it/s]

In [7]:
italian_original_features_df = getFeaturesFromAudioFiles(italian_pd_audiofiles_path, italian_control_audiofiles_path)


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/219 [00:00<?, ?it/s]

In [8]:
italian_outname = 'italian_original_features.csv'
mdvr_outname = 'mdvr_original_featuers.csv'

outdir = '../data/tables/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

italian_original_features_df.to_csv(os.path.join(outdir, italian_outname),index=False)
mdvr_original_featuers_df.to_csv(os.path.join(outdir, mdvr_outname),index=False)